In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from urllib2 import urlopen
from pandas.io.json import json_normalize 
from oauth2client.service_account import ServiceAccountCredentials
import gspread

#Client Login
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json',scope)
client = gspread.authorize(creds)
#Sheets to use
sheet2 = client.open("Scraping").worksheet('30Days')
sheet = client.open("Scraping").worksheet('Coins')

#GET COIN LIST
stats = sheet.col_values(1) #select column A
del stats[0] #Do not count the title
coinlist = stats[:3] #Select top 3
print(coinlist) #Print to test it works
day30 = [] #Create empty 30 day


#GET DATA 
for item in coinlist:
	try:
		url = urlopen("https://min-api.cryptocompare.com/data/histoday?fsym="+item+"&tsym=USD&limit=30").read().decode()
		Bitter = json.loads(url)
		df = json_normalize(Bitter['Data'])
		df2 = pd.DataFrame(df['close'])
		day30.append(item)
		for row in df2['close']:
			
			day30.append(row)
		print("Added "+item)
	except KeyError:
		print("mistakes happen: "+item+" take a look at this Ced")

#DECLARE CELLS TO BE UPDATED
x = 0
cell_list = sheet2.range('A1:AF20')
for cell in cell_list:
    cell.value = day30[x]
    x = x + 1
    if x >= len(day30):
        break;

# Update in batch
sheet2.update_cells(cell_list)
print("Finished Updating")

In [ ]:
import os
os.system("scp caflores@ltx1-holdemgw01.grid.linkedin.com:/export/home/caflores/edu.csv ~/Desktop/")

In [ ]:
import os
os.system("scp caflores@lva1-insights01.linkedin.biz:/export/apps/lms_cf_projects/iceCream/iceCreamFlavors.csv ~/Desktop/")

In [ ]:
import os
os.system("ssh -k lva1-insights01.linkedin.biz")